Contents 

1. Import data 
2. Wrangle Data
3. Conduct quality and consistency checks
4. Combine customer data with the rest of the prepared Instacart data
5. Export DF as pickle file

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#define path 
path = '/Users/wenboyang/Desktop/Data Analytics/05-08-2023 Instacart Basket Analysis /02 Data/'

1. Import data

In [3]:
#import customer data 
df = pd.read_csv(os.path.join(path, 'Original Data','customers.csv'))

In [4]:
# Import orders_products_merged.pkl 
ords_prods_merge = pd.read_pickle(os.path.join(path, 'Prepared Data', 'orders_products_merged_updated_2.pkl'))

2. Wrangle Data 

In [5]:
df.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [6]:
#rename columns 
df.rename(columns={'Surnam': 'last_name'}, inplace=True)
df.rename(columns={'First Name': 'first_name'}, inplace=True)
df.rename(columns={'Gender':'gender'}, inplace=True)
df.rename(columns={'Age':'age'}, inplace=True)
df.rename(columns={'STATE':'state'}, inplace=True)
df.rename(columns={'date_joined': 'date_joined'}, inplace=True)
df.rename(columns={'n_dependants': 'dependents'}, inplace=True)
df.rename(columns={'fam_status': 'marital_status'}, inplace=True)
df.rename(columns={'income': 'income'}, inplace=True)

In [7]:
df.head()

,user_id,first_name,last_name,gender,state,age,date_joined,dependents,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


3. Quality and Consistency Checks 

In [8]:
df.dtypes

user_id            int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
dependents         int64
marital_status    object
income             int64
dtype: object

In [9]:
#change user_id to string 
df['user_id'] = df['user_id'].astype('str')

In [10]:
df.dtypes

user_id           object
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
dependents         int64
marital_status    object
income             int64
dtype: object

In [11]:
#perform descriptive statistics 
df.describe()

,age,dependents,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


In [12]:
#check for missing values 
df.isnull().sum()

user_id               0
first_name        11259
last_name             0
gender                0
state                 0
age                   0
date_joined           0
dependents            0
marital_status        0
income                0
dtype: int64

In [13]:
# Create new df for where First Name column is blank
df_nan = df[df["first_name"].isnull() == True]

I will keep these rows because they contain other useful information even though first name is missing 

In [14]:
#Check for data consistency 
for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)
  else:
    print ('No mixed dtypes', col)

No mixed dtypes user_id
first_name
No mixed dtypes last_name
No mixed dtypes gender
No mixed dtypes state
No mixed dtypes age
No mixed dtypes date_joined
No mixed dtypes dependents
No mixed dtypes marital_status
No mixed dtypes income


In [15]:
#Change all First Name to string 
df['First Name'] = df['first_name'].astype('str')

In [16]:
#Run check 
for col in df.columns.tolist():
  weird = (df[[col]].applymap(type) != df[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df[weird]) > 0:
    print (col)
  else:
    print ('No mixed dtypes', col)

No mixed dtypes user_id
first_name
No mixed dtypes last_name
No mixed dtypes gender
No mixed dtypes state
No mixed dtypes age
No mixed dtypes date_joined
No mixed dtypes dependents
No mixed dtypes marital_status
No mixed dtypes income
No mixed dtypes First Name


In [17]:
#Check missing values 
df.isnull().sum()

user_id               0
first_name        11259
last_name             0
gender                0
state                 0
age                   0
date_joined           0
dependents            0
marital_status        0
income                0
First Name            0
dtype: int64

No more missing values because the data type for First Name was updated

In [18]:
#Check for duplicates 
duplicates = df.duplicated()

In [19]:
print(df[duplicates])

Empty DataFrame
Columns: [user_id, first_name, last_name, gender, state, age, date_joined, dependents, marital_status, income, First Name]
Index: []


No duplicates

4.Combine customer data with the rest of your prepared Instacart data

In [20]:
#Make sure key columms are same data type 
ords_prods_merge.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
price_range_loc             object
busiest day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
mean_price                 float64
spending_flag               object
mean_freq                  float64
freq_flag                   object
dtype: object

In [21]:
df.dtypes

user_id           object
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
dependents         int64
marital_status    object
income             int64
First Name        object
dtype: object

In [22]:
#Change user_id to object 
ords_prods_merge['user_id'] = ords_prods_merge['user_id'].astype('str')

In [23]:
ords_prods_merge.dtypes

order_id                     int64
user_id                     object
order_number                 int64
orders_day_of_week           int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
price_range_loc             object
busiest day                 object
busiest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
mean_price                 float64
spending_flag               object
mean_freq                  float64
freq_flag                   object
dtype: object

In [24]:
#merge the two dfs based on 'user_id'
df_combined = ords_prods_merge.merge(df, on = 'user_id', '')

In [25]:
df_combined.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,first_name,last_name,gender,state,age,date_joined,dependents,marital_status,income,First Name
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,Linda
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,Linda
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,Linda
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,Linda
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,Linda


5. Export df

In [26]:
df_combined.to_pickle(os.path.join(path, 'Prepared Data', 'customer_merged.pkl'))

In [27]:
crosstab = pd.crosstab(ords_prods_merge['days_since_prior_order'], ords_prods_merge['order_number'], dropna = False)

In [28]:
crosstab.to_clipboard()